# CONTINUAL LEARNING
**OPTION** : MNIST
**MEMBERS** : Shivakalyan Soundarathiagarajan and Kowsick Venkatachalapathi

**Gradient Episodic Memory**

1.Continual learning, where the model observes, once and one by one, examples concerning a sequence of tasks.

2.Set of metrics to evaluate models learning over a continuum of data. 

3.Gradient Episodic Memory (GEM) alleviates forgetting, while allowing beneficial transfer of knowledge to previous tasks. Experiments on variants of the MNIST and CIFAR-100 datasets demonstrate the strong performance of GEM when compared to the state-of-the-art.  

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
MY_PYTHON="!python" #TODO: modified to add
MNIST_ROTA="--n_layers 2 --n_hiddens 100 --data_path data/ --save_path results/ --batch_size 10 --log_every 100 --samples_per_task 1000 --data_file mnist_rotations.pt    --cuda no  --seed 0"
MNIST_PERM="--n_layers 2 --n_hiddens 100 --data_path data/ --save_path results/ --batch_size 10 --log_every 100 --samples_per_task 1000 --data_file mnist_permutations.pt --cuda no  --seed 0"
CIFAR_100i="--n_layers 2 --n_hiddens 100 --data_path data/ --save_path results/ --batch_size 10 --log_every 100 --samples_per_task 2500 --data_file cifar100.pt           --cuda yes --seed 0"  # TODO: MODIFIED CHECK


In [4]:
%cd /content/drive/My\ Drive/DL

/content/drive/My Drive/DL


In [3]:
# build datasets
%cd data/
%cd raw/

/content/drive/My Drive/DL/data
/content/drive/My Drive/DL/data/raw


# DOWNLOADING CIFAR and split in to MNIST TRAIN AND TEST DATASET
# Executing Raw.py


*   Downloading Data - cifar-100-python.tar.gz,  mnist.npz
*   Output mnist_train.pt and mnist_test.pt



In [ ]:
import numpy as np
import subprocess
import pickle
import torch
import os

cifar_path = "cifar-100-python.tar.gz"
mnist_path = "mnist.npz"

# URL from: https://www.cs.toronto.edu/~kriz/cifar.html
if not os.path.exists(cifar_path):
    subprocess.call("wget https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz", shell=True)

subprocess.call("tar xzfv cifar-100-python.tar.gz", shell=True)

# URL from: https://github.com/fchollet/keras/blob/master/keras/datasets/mnist.py
if not os.path.exists(mnist_path):
    subprocess.call("wget https://s3.amazonaws.com/img-datasets/mnist.npz", shell=True)

def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

cifar100_train = unpickle('cifar-100-python/train')
cifar100_test = unpickle('cifar-100-python/test')

x_tr = torch.from_numpy(cifar100_train[b'data'])
y_tr = torch.LongTensor(cifar100_train[b'fine_labels'])
x_te = torch.from_numpy(cifar100_test[b'data'])
y_te = torch.LongTensor(cifar100_test[b'fine_labels'])

torch.save((x_tr, y_tr, x_te, y_te), 'cifar100.pt')

f = np.load('mnist.npz')
x_tr = torch.from_numpy(f['x_train'])
y_tr = torch.from_numpy(f['y_train']).long()
x_te = torch.from_numpy(f['x_test'])
y_te = torch.from_numpy(f['y_test']).long()
f.close()

torch.save((x_tr, y_tr), 'mnist_train.pt')
torch.save((x_te, y_te), 'mnist_test.pt')


In [ ]:
#import raw
!python raw.py

# DOWNLOADING MNIST PERMUTATIONS

In [4]:
%cd ..

/content/drive/My Drive/DL/data


In [ ]:
import argparse
import os.path
import torch

parser = argparse.ArgumentParser()

parser.add_argument('--i', default='raw/', help='input directory')
parser.add_argument('--o', default='mnist_permutations.pt', help='output file')
parser.add_argument('--n_tasks', default=3, type=int, help='number of tasks')
parser.add_argument('--seed', default=0, type=int, help='random seed')
args = parser.parse_args(args=[]) #TODO: MODIFIED CHECK

torch.manual_seed(args.seed)

tasks_tr = []
tasks_te = []

x_tr, y_tr = torch.load(os.path.join(args.i, 'mnist_train.pt'))
x_te, y_te = torch.load(os.path.join(args.i, 'mnist_test.pt'))
x_tr = x_tr.float().view(x_tr.size(0), -1) / 255.0
x_te = x_te.float().view(x_te.size(0), -1) / 255.0
y_tr = y_tr.view(-1).long()
y_te = y_te.view(-1).long()

for t in range(args.n_tasks):
    p = torch.randperm(x_tr.size(1)).long().view(-1)

    tasks_tr.append(['random permutation', x_tr.index_select(1, p), y_tr])
    tasks_te.append(['random permutation', x_te.index_select(1, p), y_te])

torch.save([tasks_tr, tasks_te], args.o)


In [ ]:
#pwd
$MY_PYTHON mnist_permutations.py \
	--o mnist_permutations.pt \
	--seed 0 \
	--n_tasks 20

# DOWNLOADING MNIST ROTATIONS

In [ ]:
from torchvision import transforms
from PIL import Image
import argparse
import os.path
import random
import torch


def rotate_dataset(d, rotation):
    result = torch.FloatTensor(d.size(0), 784)
    tensor = transforms.ToTensor()

    for i in range(d.size(0)):
        img = Image.fromarray(d[i].numpy(), mode='L')
        result[i] = tensor(img.rotate(rotation)).view(784)
    return result


parser = argparse.ArgumentParser()

parser.add_argument('--i', default='raw/', help='input directory')
parser.add_argument('--o', default='mnist_rotations.pt', help='output file')
parser.add_argument('--n_tasks', default=10, type=int, help='number of tasks')
parser.add_argument('--min_rot', default=0.,
                    type=float, help='minimum rotation')
parser.add_argument('--max_rot', default=90.,
                    type=float, help='maximum rotation')
parser.add_argument('--seed', default=0, type=int, help='random seed')

args = parser.parse_args(args=[]) #TODO : MODIFIED CHECK

torch.manual_seed(args.seed)

tasks_tr = []
tasks_te = []

x_tr, y_tr = torch.load(os.path.join(args.i, 'mnist_train.pt'))
x_te, y_te = torch.load(os.path.join(args.i, 'mnist_test.pt'))

for t in range(args.n_tasks):
    min_rot = 1.0 * t / args.n_tasks * (args.max_rot - args.min_rot) + \
        args.min_rot
    max_rot = 1.0 * (t + 1) / args.n_tasks * \
        (args.max_rot - args.min_rot) + args.min_rot
    rot = random.random() * (max_rot - min_rot) + min_rot

    tasks_tr.append([rot, rotate_dataset(x_tr, rot), y_tr])
    tasks_te.append([rot, rotate_dataset(x_te, rot), y_te])

torch.save([tasks_tr, tasks_te], args.o)


In [ ]:
!python mnist_rotations.py \
	--o mnist_rotations.pt\
	--seed 0 \
	--min_rot 0 \
	--max_rot 180 \
	--n_tasks 20

# PROCESSING CIFAR100

In [ ]:
import argparse
import os.path
import torch

parser = argparse.ArgumentParser()

parser.add_argument('--i', default='raw/cifar100.pt', help='input directory')
parser.add_argument('--o', default='cifar100.pt', help='output file')
parser.add_argument('--n_tasks', default=10, type=int, help='number of tasks')
parser.add_argument('--seed', default=0, type=int, help='random seed')
args = parser.parse_args(args=[]) #TODO : MODIFIED CHECK

torch.manual_seed(args.seed)

tasks_tr = []
tasks_te = []

x_tr, y_tr, x_te, y_te = torch.load(os.path.join(args.i))
x_tr = x_tr.float().view(x_tr.size(0), -1) / 255.0
x_te = x_te.float().view(x_te.size(0), -1) / 255.0

cpt = int(100 / args.n_tasks)

for t in range(args.n_tasks):
    c1 = t * cpt
    c2 = (t + 1) * cpt
    i_tr = ((y_tr >= c1) & (y_tr < c2)).nonzero().view(-1)
    i_te = ((y_te >= c1) & (y_te < c2)).nonzero().view(-1)
    tasks_tr.append([(c1, c2), x_tr[i_tr].clone(), y_tr[i_tr].clone()])
    tasks_te.append([(c1, c2), x_te[i_te].clone(), y_te[i_te].clone()])

torch.save([tasks_tr, tasks_te], args.o)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)


In [ ]:
!python cifar100.py \
	--o cifar100.pt \
	--seed 0 \
	--n_tasks 20

cifar100.py:33: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  i_tr = ((y_tr >= c1) & (y_tr < c2)).nonzero().view(-1)


In [5]:
%pwd

'/content/drive/My Drive/DL'

In [13]:
%cd metrics

/content/drive/My Drive/DL/metrics


In [14]:
!python metrics.py

In [18]:
%pwd

'/content/drive/My Drive/DL/metrics'

In [19]:
%cd ..

/content/drive/My Drive/DL


# MAIN : STOCHASTIC GRADIENT DESCENT - CONTINUAL LEARNING
# MODELS


1.   Single
2.   Multimodal
3. independent
4.ewc - Elastic Weight Consolidation
5.gem - Gradient Episodic Memory



In [21]:
import importlib
import datetime
import argparse
import random
import uuid
import time
import os

import numpy as np

import torch
from metrics import confusion_matrix #TODO : MODIFIED

# continuum iterator #########################################################


def load_datasets(args):
    d_tr, d_te = torch.load(args.data_path + '/' + args.data_file)
    n_inputs = d_tr[0][1].size(1)
    n_outputs = 0
    for i in range(len(d_tr)):
        n_outputs = max(n_outputs, d_tr[i][2].max().item())
        n_outputs = max(n_outputs, d_te[i][2].max().item())
    return d_tr, d_te, n_inputs, n_outputs + 1, len(d_tr)


class Continuum:

    def __init__(self, data, args):
        self.data = data
        self.batch_size = args.batch_size
        n_tasks = len(data)
        task_permutation = range(n_tasks)

        if args.shuffle_tasks == 'yes':
            task_permutation = torch.randperm(n_tasks).tolist()

        sample_permutations = []

        for t in range(n_tasks):
            N = data[t][1].size(0)
            if args.samples_per_task <= 0:
                n = N
            else:
                n = min(args.samples_per_task, N)

            p = torch.randperm(N)[0:n]
            sample_permutations.append(p)

        self.permutation = []

        for t in range(n_tasks):
            task_t = task_permutation[t]
            for _ in range(args.n_epochs):
                task_p = [[task_t, i] for i in sample_permutations[task_t]]
                random.shuffle(task_p)
                self.permutation += task_p

        self.length = len(self.permutation)
        self.current = 0

    def __iter__(self):
        return self

    def next(self):
        return self.__next__()

    def __next__(self):
        if self.current >= self.length:
            raise StopIteration
        else:
            ti = self.permutation[self.current][0]
            j = []
            i = 0
            while (((self.current + i) < self.length) and
                   (self.permutation[self.current + i][0] == ti) and
                   (i < self.batch_size)):
                j.append(self.permutation[self.current + i][1])
                i += 1
            self.current += i
            j = torch.LongTensor(j)
            return self.data[ti][1][j], ti, self.data[ti][2][j]

# train handle ###############################################################


def eval_tasks(model, tasks, args):
    model.eval()
    result = []
    for i, task in enumerate(tasks):
        t = i
        x = task[1]
        y = task[2]
        rt = 0
        
        eval_bs = x.size(0)

        for b_from in range(0, x.size(0), eval_bs):
            b_to = min(b_from + eval_bs, x.size(0) - 1)
            if b_from == b_to:
                xb = x[b_from].view(1, -1)
                yb = torch.LongTensor([y[b_to]]).view(1, -1)
            else:
                xb = x[b_from:b_to]
                yb = y[b_from:b_to]
            if args.cuda:
                xb = xb.cuda()
            _, pb = torch.max(model(xb, t).data.cpu(), 1, keepdim=False)
            rt += (pb == yb).float().sum()

        result.append(rt / x.size(0))

    return result


def life_experience(model, continuum, x_te, args):
    result_a = []
    result_t = []

    current_task = 0
    time_start = time.time()

    for (i, (x, t, y)) in enumerate(continuum):
        if(((i % args.log_every) == 0) or (t != current_task)):
            result_a.append(eval_tasks(model, x_te, args))
            result_t.append(current_task)
            current_task = t

        v_x = x.view(x.size(0), -1)
        v_y = y.long()

        if args.cuda:
            v_x = v_x.cuda()
            v_y = v_y.cuda()

        model.train()
        model.observe(v_x, t, v_y)

    result_a.append(eval_tasks(model, x_te, args))
    result_t.append(current_task)

    time_end = time.time()
    time_spent = time_end - time_start

    return torch.Tensor(result_t), torch.Tensor(result_a), time_spent


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Continuum learning')

    # model parameters
    parser.add_argument('--model', type=str, default='single',
                        help='model to train')
    parser.add_argument('--n_hiddens', type=int, default=100,
                        help='number of hidden neurons at each layer')
    parser.add_argument('--n_layers', type=int, default=2,
                        help='number of hidden layers')

    # memory parameters
    parser.add_argument('--n_memories', type=int, default=0,
                        help='number of memories per task')
    parser.add_argument('--memory_strength', default=0, type=float,
                        help='memory strength (meaning depends on memory)')
    parser.add_argument('--finetune', default='no', type=str,
                        help='whether to initialize nets in indep. nets')

    # optimizer parameters
    parser.add_argument('--n_epochs', type=int, default=1,
                        help='Number of epochs per task')
    parser.add_argument('--batch_size', type=int, default=10,
                        help='batch size')
    parser.add_argument('--lr', type=float, default=1e-3,
                        help='SGD learning rate')

    # experiment parameters
    parser.add_argument('--cuda', type=str, default='no',
                        help='Use GPU?')
    parser.add_argument('--seed', type=int, default=0,
                        help='random seed')
    parser.add_argument('--log_every', type=int, default=100,
                        help='frequency of logs, in minibatches')
    parser.add_argument('--save_path', type=str, default='results/',
                        help='save models at the end of training')

    # data parameters
    parser.add_argument('--data_path', default='data/',
                        help='path where data is located')
    parser.add_argument('--data_file', default='mnist_permutations.pt',
                        help='data file')
    parser.add_argument('--samples_per_task', type=int, default=-1,
                        help='training samples per task (all if negative)')
    parser.add_argument('--shuffle_tasks', type=str, default='no',
                        help='present tasks in order')
    args = parser.parse_args(args=[]) # TODO : MODIFIED CHECK

    args.cuda = True if args.cuda == 'yes' else False
    args.finetune = True if args.finetune == 'yes' else False

    # multimodal model has one extra layer
    if args.model == 'multimodal':
        args.n_layers -= 1

    # unique identifier
    uid = uuid.uuid4().hex

    # initialize seeds
    torch.backends.cudnn.enabled = False
    torch.manual_seed(args.seed)
    np.random.seed(args.seed)
    random.seed(args.seed)
    if args.cuda:
        torch.cuda.manual_seed_all(args.seed)

    # load data
    x_tr, x_te, n_inputs, n_outputs, n_tasks = load_datasets(args)

    # set up continuum
    continuum = Continuum(x_tr, args)

    # load model
    Model = importlib.import_module('model.' + args.model)
    model = Model.Net(n_inputs, n_outputs, n_tasks, args)
    if args.cuda:
        model.cuda()

    # run model on continuum
    result_t, result_a, spent_time = life_experience(
        model, continuum, x_te, args)

    # prepare saving path and file name
    if not os.path.exists(args.save_path):
        os.makedirs(args.save_path)

    fname = args.model + '_' + args.data_file + '_'
    fname += datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    fname += '_' + uid
    fname = os.path.join(args.save_path, fname)

    # save confusion matrix and print one line of stats
    stats = confusion_matrix(result_t, result_a, fname + '.txt')
    one_liner = str(vars(args)) + ' # '
    one_liner += ' '.join(["%.3f" % stat for stat in stats])
    print(fname + ': ' + one_liner + ' # ' + str(spent_time))

    # save all results in binary file
    torch.save((result_t, result_a, model.state_dict(),
                stats, one_liner, args), fname + '.pt')


results/single_mnist_permutations.pt_2020_10_17_13_23_18_a0059f10775e4a0a88ecf6f6e063916e: {'model': 'single', 'n_hiddens': 100, 'n_layers': 2, 'n_memories': 0, 'memory_strength': 0, 'finetune': False, 'n_epochs': 1, 'batch_size': 10, 'lr': 0.001, 'cuda': False, 'seed': 0, 'log_every': 100, 'save_path': 'results/', 'data_path': 'data/', 'data_file': 'mnist_permutations.pt', 'samples_per_task': -1, 'shuffle_tasks': 'no'} # 0.832 -0.049 -0.010 # 39.38511252403259


# CONTINUAL LEARNING : SINGLE MODEL

In [23]:
!python main.py $MNIST_ROTA --model single --lr 0.003

results/single_mnist_rotations.pt_2020_10_17_13_24_39_8f3035f3bee44f1aa3c627e3c5d9dbc6: {'model': 'single', 'n_hiddens': 100, 'n_layers': 2, 'n_memories': 0, 'memory_strength': 0, 'finetune': False, 'n_epochs': 1, 'batch_size': 10, 'lr': 0.003, 'cuda': False, 'seed': 0, 'log_every': 100, 'save_path': 'results/', 'data_path': 'data/', 'data_file': 'mnist_rotations.pt', 'samples_per_task': 1000, 'shuffle_tasks': 'no'} # 0.574 0.086 0.276 # 5.51280403137207


In [ ]:
!python main.py $MNIST_PERM --model single --lr 0.03

results/single_mnist_permutations.pt_2020_10_07_01_46_29_fedc5f4277e748f78d5c439342ba728b: {'model': 'single', 'n_hiddens': 100, 'n_layers': 2, 'n_memories': 0, 'memory_strength': 0, 'finetune': False, 'n_epochs': 1, 'batch_size': 10, 'lr': 0.03, 'cuda': False, 'seed': 0, 'log_every': 100, 'save_path': 'results/', 'data_path': 'data/', 'data_file': 'mnist_permutations.pt', 'samples_per_task': 1000, 'shuffle_tasks': 'no'} # 0.724 -0.031 -0.017 # 0.8295390605926514


In [ ]:
!python main.py $CIFAR_100i --model single --lr 1.0

Traceback (most recent call last):
  File "main.py", line 234, in <module>
    model, continuum, x_te, args)
  File "main.py", line 145, in life_experience
    result_a.append(eval_tasks(model, x_te, args))
  File "main.py", line 114, in eval_tasks
    _, pb = torch.max(model(xb, t).data.cpu(), 1, keepdim=False)
  File "/usr/local/lib/python3.6/dist-packages/torch/nn/modules/module.py", line 722, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/content/drive/My Drive/DL/model/single.py", line 50, in forward
    output = self.net(x)
  File "/usr/local/lib/python3.6/dist-packages/torch/nn/modules/module.py", line 722, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/content/drive/My Drive/DL/model/common.py", line 97, in forward
    out = self.layer4(out)
  File "/usr/local/lib/python3.6/dist-packages/torch/nn/modules/module.py", line 722, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/torch/nn/

In [26]:
pip install quadprog

In [37]:
#!python commmon.py
%pwd
%cd '/content/drive/My Drive/DL/model'

/content/drive/My Drive/DL/model


In [38]:
!python common.py

In [34]:
# Copyright 2017-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.

import math
import torch
import torch.nn as nn
from torch.nn.functional import relu, avg_pool2d


def Xavier(m):
    if m.__class__.__name__ == 'Linear':
        fan_in, fan_out = m.weight.data.size(1), m.weight.data.size(0)
        std = 1.0 * math.sqrt(2.0 / (fan_in + fan_out))
        a = math.sqrt(3.0) * std
        m.weight.data.uniform_(-a, a)
        m.bias.data.fill_(0.0)


class MLP(nn.Module):
    def __init__(self, sizes):
        super(MLP, self).__init__()
        layers = []

        for i in range(0, len(sizes) - 1):
            layers.append(nn.Linear(sizes[i], sizes[i + 1]))
            if i < (len(sizes) - 2):
                layers.append(nn.ReLU())

        self.net = nn.Sequential(*layers)
        self.net.apply(Xavier)

    def forward(self, x):
        return self.net(x)


def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(in_planes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1,
                          stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes, nf):
        super(ResNet, self).__init__()
        self.in_planes = nf

        self.conv1 = conv3x3(3, nf * 1)
        self.bn1 = nn.BatchNorm2d(nf * 1)
        self.layer1 = self._make_layer(block, nf * 1, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, nf * 2, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, nf * 4, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, nf * 8, num_blocks[3], stride=2)
        self.linear = nn.Linear(nf * 8 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        bsz = x.size(0)
        out = relu(self.bn1(self.conv1(x.view(bsz, 3, 32, 32))))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18(nclasses, nf=20):
    return ResNet(BasicBlock, [2, 2, 2, 2], nclasses, nf)


In [40]:
%cd ..

/content/drive/My Drive/DL


In [47]:
%cd ../model

/content/drive/My Drive/DL/model


In [51]:
%cd ..

/content/drive/My Drive/DL


# Continual learning : EWC


In [52]:
!python main.py $MNIST_ROTA --model ewc --lr 0.01 --n_memories 1000 --memory_strength 1000

results/ewc_mnist_rotations.pt_2020_10_17_13_40_04_634158387df741a5987b9fa4376669f0: {'model': 'ewc', 'n_hiddens': 100, 'n_layers': 2, 'n_memories': 1000, 'memory_strength': 1000.0, 'finetune': False, 'n_epochs': 1, 'batch_size': 10, 'lr': 0.01, 'cuda': False, 'seed': 0, 'log_every': 100, 'save_path': 'results/', 'data_path': 'data/', 'data_file': 'mnist_rotations.pt', 'samples_per_task': 1000, 'shuffle_tasks': 'no'} # 0.624 -0.107 0.495 # 13.038573980331421


In [53]:
!python main.py $MNIST_PERM --model ewc --lr 0.1  --n_memories 10   --memory_strength 3

results/ewc_mnist_permutations.pt_2020_10_17_13_40_28_69460d616f9440199d92ef8a7a27e9f8: {'model': 'ewc', 'n_hiddens': 100, 'n_layers': 2, 'n_memories': 10, 'memory_strength': 3.0, 'finetune': False, 'n_epochs': 1, 'batch_size': 10, 'lr': 0.1, 'cuda': False, 'seed': 0, 'log_every': 100, 'save_path': 'results/', 'data_path': 'data/', 'data_file': 'mnist_permutations.pt', 'samples_per_task': 1000, 'shuffle_tasks': 'no'} # 0.785 -0.026 -0.021 # 1.2530765533447266


In [ ]:
!python main.py $CIFAR_100i --model ewc --lr 1.0  --n_memories 10   --memory_strength 1

In [57]:
%cd ..

/content/drive/My Drive/DL


# MULTI MODAL

In [58]:
# model "multimodal"
!python main.py $MNIST_ROTA  --model multimodal --lr 0.1
!python main.py $MNIST_PERM  --model multimodal --lr 0.1

results/multimodal_mnist_rotations.pt_2020_10_17_13_57_31_0f33994c2eae4d34b7b4da279269c6d3: {'model': 'multimodal', 'n_hiddens': 100, 'n_layers': 1, 'n_memories': 0, 'memory_strength': 0, 'finetune': False, 'n_epochs': 1, 'batch_size': 10, 'lr': 0.1, 'cuda': False, 'seed': 0, 'log_every': 100, 'save_path': 'results/', 'data_path': 'data/', 'data_file': 'mnist_rotations.pt', 'samples_per_task': 1000, 'shuffle_tasks': 'no'} # 0.787 0.000 0.023 # 6.4455726146698
results/multimodal_mnist_permutations.pt_2020_10_17_13_57_35_068542a896874896972e5ff2d49bd9c7: {'model': 'multimodal', 'n_hiddens': 100, 'n_layers': 1, 'n_memories': 0, 'memory_strength': 0, 'finetune': False, 'n_epochs': 1, 'batch_size': 10, 'lr': 0.1, 'cuda': False, 'seed': 0, 'log_every': 100, 'save_path': 'results/', 'data_path': 'data/', 'data_file': 'mnist_permutations.pt', 'samples_per_task': 1000, 'shuffle_tasks': 'no'} # 0.789 0.031 0.015 # 0.8775877952575684


# INDEPENDENT

In [59]:
# model "independent"
!python main.py $MNIST_ROTA --model independent --lr 0.1  --finetune yes 
!python main.py $MNIST_PERM --model independent --lr 0.03 --finetune yes 


results/independent_mnist_rotations.pt_2020_10_17_13_58_31_0ffd3057fb0341f99b5d75eac0643371: {'model': 'independent', 'n_hiddens': 100, 'n_layers': 2, 'n_memories': 0, 'memory_strength': 0, 'finetune': True, 'n_epochs': 1, 'batch_size': 10, 'lr': 0.1, 'cuda': False, 'seed': 0, 'log_every': 100, 'save_path': 'results/', 'data_path': 'data/', 'data_file': 'mnist_rotations.pt', 'samples_per_task': 1000, 'shuffle_tasks': 'no'} # 0.739 0.000 0.000 # 2.529115915298462
results/independent_mnist_permutations.pt_2020_10_17_13_58_34_93eee9e8bd1e44499d273d39ab71cac1: {'model': 'independent', 'n_hiddens': 100, 'n_layers': 2, 'n_memories': 0, 'memory_strength': 0, 'finetune': True, 'n_epochs': 1, 'batch_size': 10, 'lr': 0.03, 'cuda': False, 'seed': 0, 'log_every': 100, 'save_path': 'results/', 'data_path': 'data/', 'data_file': 'mnist_permutations.pt', 'samples_per_task': 1000, 'shuffle_tasks': 'no'} # 0.728 0.000 0.000 # 0.5108816623687744


In [ ]:
!python main.py $CIFAR_100i --model independent --lr 0.3  --finetune yes 

# Continual Learning : GEM

In [42]:
# model "GEM"
!python main.py $MNIST_ROTA --model gem --lr 0.1 --n_memories 256 --memory_strength 0.5


results/gem_mnist_rotations.pt_2020_10_17_13_31_27_ac1f9256f66540009881eb50bfc12dfc: {'model': 'gem', 'n_hiddens': 100, 'n_layers': 2, 'n_memories': 256, 'memory_strength': 0.5, 'finetune': False, 'n_epochs': 1, 'batch_size': 10, 'lr': 0.1, 'cuda': False, 'seed': 0, 'log_every': 100, 'save_path': 'results/', 'data_path': 'data/', 'data_file': 'mnist_rotations.pt', 'samples_per_task': 1000, 'shuffle_tasks': 'no'} # 0.893 0.044 0.618 # 35.57505917549133


In [43]:
!python main.py $MNIST_PERM --model gem --lr 0.1 --n_memories 256 --memory_strength 0.5

results/gem_mnist_permutations.pt_2020_10_17_13_31_49_0f0936a16c594a33b34b941ef91b21c5: {'model': 'gem', 'n_hiddens': 100, 'n_layers': 2, 'n_memories': 256, 'memory_strength': 0.5, 'finetune': False, 'n_epochs': 1, 'batch_size': 10, 'lr': 0.1, 'cuda': False, 'seed': 0, 'log_every': 100, 'save_path': 'results/', 'data_path': 'data/', 'data_file': 'mnist_permutations.pt', 'samples_per_task': 1000, 'shuffle_tasks': 'no'} # 0.839 0.027 -0.020 # 2.697176456451416


In [ ]:
!python main.py $CIFAR_100i --model gem --lr 0.1 --n_memories 256 --memory_strength 0.5

# RESULTS : EVALUATION

In [60]:
%cd results/

/content/drive/My Drive/DL/results


In [61]:
import matplotlib as mpl
mpl.use('Agg')

# if 'roman' in mpl.font_manager.weight_dict.keys():
#     del mpl.font_manager.weight_dict['roman']
# mpl.font_manager._rebuild()

mpl.rcParams["font.family"] = "Times New Roman"
mpl.rcParams["font.family"] = "DejaVu Serif"

from matplotlib import pyplot as plt
from glob import glob
import numpy as np
import torch

models = ['single', 'independent', 'multimodal', 'icarl', 'ewc', 'gem']
datasets = ['mnist_permutations', 'mnist_rotations', 'cifar100']

names_datasets = {'mnist_permutations': 'MNIST permutations',
                  'mnist_rotations': 'MNIST rotations',
                  'cifar100': 'CIFAR-100'}

names_models = {'single': 'single',
                'independent': 'independent',
                'multimodal': 'multimodal',
                'icarl': 'iCARL',
                'ewc': 'EWC',
                'gem': 'GEM'}

colors = {'single': 'C0',
          'independent': 'C1',
          'multimodal': 'C2',
          'icarl': 'C2',
          'ewc': 'C3',
          'gem': 'C4'}

barplot = {}

for dataset in datasets:
    barplot[dataset] = {}
    for model in models:
        barplot[dataset][model] = {}
        matches = glob(model + '*' + dataset + '*.pt')
        if len(matches):
            data = torch.load(matches[0], map_location=lambda storage, loc: storage)
            acc, bwt, fwt = data[3][:]
            barplot[dataset][model]['acc'] = acc
            barplot[dataset][model]['bwt'] = bwt
            barplot[dataset][model]['fwt'] = fwt

for dataset in datasets:
    x_lab = []
    y_acc = []
    y_bwt = []
    y_fwt = []

    for i, model in enumerate(models):
        if barplot[dataset][model] != {}:
            x_lab.append(model)
            y_acc.append(barplot[dataset][model]['acc'])
            y_bwt.append(barplot[dataset][model]['bwt'])
            y_fwt.append(barplot[dataset][model]['fwt'])

    x_ind = np.arange(len(y_acc))

    plt.figure(figsize=(7, 3))
    all_colors = []
    for xi, yi, li in zip(x_ind, y_acc, x_lab):
        plt.bar(xi, yi, label=names_models[li], color=colors[li])
        all_colors.append(colors[li])
    plt.bar(x_ind + (len(y_acc) + 1) * 1, y_bwt, color=all_colors)
    plt.bar(x_ind + (len(y_acc) + 1) * 2, y_fwt, color=all_colors)
    plt.xticks([2, 8, 14], ['ACC', 'BWT', 'FWT'], fontsize=16)
    plt.yticks(fontsize=16)
    plt.xlim(-1, len(y_acc) * 3 + 2)
    plt.ylabel('classification accuracy', fontsize=16)
    plt.title(names_datasets[dataset], fontsize=16)
    plt.legend(fontsize=12)
    plt.tight_layout()
    plt.savefig('CS677_DL_Shiva_Kowsick_barplot_%s.pdf' % dataset, bbox_inches='tight')
    # plt.show()

evoplot = {}

for dataset in datasets:
    evoplot[dataset] = {}
    for model in models:
        matches = glob(model + '*' + dataset + '*.pt')
        if len(matches):
            data = torch.load(matches[0], map_location=lambda storage, loc: storage)
            evoplot[dataset][model] = data[1][:, 0].numpy()

for dataset in datasets:

    plt.figure(figsize=(7, 3))
    for model in models:
        if model in evoplot[dataset]:
            x = np.arange(len(evoplot[dataset][model]))
            x = (x - x.min()) / (x.max() - x.min()) * 20
            plt.plot(x, evoplot[dataset][model], color=colors[model], lw=3)
            plt.xticks(range(0, 21, 2))

    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.xlabel('task number', fontsize=16)
    plt.title(names_datasets[dataset], fontsize=16)
    plt.tight_layout()
    plt.savefig('CS677_DL_Shiva_Kowsick_evoplot_%s.pdf' % dataset, bbox_inches='tight')
    # plt.show()


No handles with labels found to put in legend.


In [ ]:
!python plot_results.py

No handles with labels found to put in legend.
No handles with labels found to put in legend.
